# 第9章: ベクトル空間法 (I)
[enwiki-20150112-400-r10-105752.txt.bz2](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2)は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパス[enwiki-20150112-400-r100-10576.txt.bz2](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r100-10576.txt.bz2)を用いよ．

In [1]:
! curl -o enwiki-20150112-400-r100-10576.txt.bz2 http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r100-10576.txt.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.0M  100 21.0M    0     0   256k      0  0:01:23  0:01:23 --:--:--  368k


In [2]:
# ! curl -o enwiki-20150112-400-r10-105752.txt.bz2 http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2

## 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．

- トークンの先頭と末尾に出現する次の文字を削除: `.,!?;:()[]'"`
- 空文字列となったトークンは削除

以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [3]:
import bz2

In [4]:
with bz2.open("enwiki-20150112-400-r100-10576.txt.bz2", mode="rt") as f:
    text = f.readlines()

In [5]:
# with bz2.open("enwiki-20150112-400-r10-105752.txt.bz2", mode="rt") as f:
#     text = f.readlines()

In [6]:
text[:5]

['Anarchism\n',
 '\n',
 'Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions, but that several authors have defined as more specific institutions based on non-hierarchical free associations. Anarchism holds the state to be undesirable, unnecessary, or harmful. While anti-statism is central, anarchism entails opposing authority or hierarchical organisation in the conduct of human relations, including, but not limited to, the state system.\n',
 'As a subtle and anti-dogmatic philosophy, anarchism draws on many currents of thought and strategy. Anarchism does not offer a fixed body of doctrine from a single particular world view, instead fluxing and flowing as a philosophy. There are many types and traditions of anarchism, not all of which are mutually exclusive. Anarchist schools of thought can differ fundamentally, supporting anything from extreme individualism to complete collectivism. Strains of anarchism have of

In [7]:
lines = []

for line in text:
    words = [word.strip().strip(".,!?;:()[]'\"") for word in line.strip().split(" ")]
    lines.append(" ".join([word for word in words if word != ""]))

In [8]:
lines[:5]

['Anarchism',
 '',
 'Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system',
 'As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into th

In [9]:
with open("output_80.txt", mode="w") as f:
    f.write("\n".join(lines))

In [10]:
! head -n 5 output_80.txt

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories o

## 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv("english_country_name.csv")

In [13]:
df

,国・地域名,英語名,数,三字,二字,場所,各行政区分
0,アイスランド,Iceland,352,ISL,IS,北ヨーロッパ,ISO 3166-2:IS
1,アイルランド,Ireland,372,IRL,IE,西ヨーロッパ,ISO 3166-2:IE
2,アゼルバイジャン,Azerbaijan,31,AZE,AZ,東ヨーロッパ,ISO 3166-2:AZ
3,アフガニスタン,Afghanistan,4,AFG,AF,中東,ISO 3166-2:AF
4,アメリカ合衆国,United States,840,USA,US,北アメリカ,ISO 3166-2:US
5,アメリカ領ヴァージン諸島,"Virgin Islands, U.S.",850,VIR,VI,中央アメリカ,ISO 3166-2:VI
6,アメリカ領サモア,American Samoa,16,ASM,AS,オセアニア,ISO 3166-2:AS
7,アラブ首長国連邦,United Arab Emirates,784,ARE,AE,中東,ISO 3166-2:AE
8,アルジェリア,Algeria,12,DZA,DZ,北アフリカ,ISO 3166-2:DZ
9,アルゼンチン,Argentina,32,ARG,AR,南アメリカ,ISO 3166-2:AR


In [14]:
country_names = set([" ".join([part.strip().strip(".,!?;:()[]'\"") for part in country_name.split(" ")]) for country_name in df["英語名"].to_list()])
country_names

{'Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antarctica',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia Plurinational State of',
 'Bonaire Saint Eustatius and Saba',
 'Bosnia and Herzegovina',
 'Botswana',
 'Bouvet Island',
 'Brazil',
 'British Indian Ocean Territory',
 'Brunei Darussalam',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Christmas Island',
 'Cocos Keeling Islands',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo the Democratic Republic of the',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 "Côte d'Ivoire",
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'E

In [15]:
with open("output_80.txt", mode="r") as f:
     text = f.readlines()

In [16]:
replaced_lines = []

for line in text:
    line = line.strip()
    for country in country_names:
        if country in line:
            line = line.replace(country, country.replace(" ", "_"))
    replaced_lines.append(line)

In [17]:
replaced_lines[:5]

['Anarchism',
 '',
 'Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system',
 'As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into th

In [18]:
with open("output_81.txt", mode="w") as f:
    f.write("\n".join(replaced_lines))

In [19]:
! head -n 5 output_81.txt

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system
As a subtle and anti-dogmatic philosophy anarchism draws on many currents of thought and strategy Anarchism does not offer a fixed body of doctrine from a single particular world view instead fluxing and flowing as a philosophy There are many types and traditions of anarchism not all of which are mutually exclusive Anarchist schools of thought can differ fundamentally supporting anything from extreme individualism to complete collectivism Strains of anarchism have often been divided into the categories o

## 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語tに関して，単語tと文脈語cのペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

- ある単語$t$の前後$d$単語を文脈語$c$として抽出する（ただし，文脈語に単語$t$そのものは含まない）
- 単語$t$を選ぶ度に，文脈幅$d$は${1,2,3,4,5}$の範囲でランダムに決める．

In [20]:
with open("output_81.txt", mode="r") as f:
     text = f.readlines()

In [21]:
from random import randint

In [22]:
f = open("output_82.tsv", mode="w")

for line in text:
    words = line.strip().split(" ")
    words_num = len(words)
    for i in range(words_num):
        d = randint(1, 5)
        for j in range(-d, d, 1):
            if j == 0:
                continue
            if 0 <= i + j < words_num:
                f.write("%s\t%s\n" % (words[i], words[i + j]))
f.close()

In [23]:
! head -n 5 output_82.tsv

Anarchism	is
is	Anarchism
a	is
political	Anarchism
political	is


## 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

- $f(t,c)$: 単語$t$と文脈語$c$の共起回数
- $f(t,∗)$: 単語$t$の出現回数
- $f(∗,c)$: 文脈語$c$の出現回数
- $N$: 単語と文脈語のペアの総出現回数

In [24]:
with open("output_82.tsv", mode="r") as f:
     text = f.readlines()

In [25]:
from collections import Counter

In [26]:
t_counter, c_counter, tc_counter = Counter(), Counter(), Counter()

In [27]:
for batch_start in range(0, len(text), 10000):
    batch_end = batch_start + 10000 if batch_start + 10000 < len(text) else len(text)
    t_list, c_list, tc_list  = [], [], []
    for line in text[batch_start:batch_end]:
        try:
            tc = line.strip().split("\t")
            t_list.append(tc[0])
            c_list.append(tc[1])
            tc_list.append(line.strip())
        except IndexError:
            print(tc)
    t_counter.update(t_list)
    c_counter.update(c_list)
    tc_counter.update(tc_list)

In [28]:
import joblib

In [29]:
joblib.dump(t_counter, "t_counter.pcl", compress=3)
joblib.dump(c_counter, "c_counter.pcl", compress=3)
joblib.dump(tc_counter, "tc_counter.pcl", compress=3)

['tc_counter.pcl']

In [30]:
N = len(text)
N

56808555

## 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列Xを作成せよ．ただし，行列Xの各要素Xtcは次のように定義する．

$f(t,c)≥10$ならば，$X_{tc}=PPMI(t,c)=max{\log \frac{N×f(t,c)}{f(t,∗)×f(∗,c)},0}$

$f(t,c)<10$ならば，$X_{tc}=0$

ここで，PPMI(t,c)はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列Xの行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列Xのほとんどの要素は0になるので，非0の要素だけを書き出せばよい．

In [31]:
from scipy import sparse

In [32]:
from collections import OrderedDict

In [33]:
size_t = len(t_counter)
size_c = len(c_counter)
matrix_x = sparse.lil_matrix((size_t, size_c))

In [34]:
ordered_t_index = OrderedDict([(t, i) for i, t in enumerate(t_counter)])
ordered_c_index = OrderedDict([(c, i) for i, c in enumerate(c_counter)])
ordered_tc_counter = OrderedDict(tc_counter)

In [35]:
import math

In [40]:
for tc in tc_counter.keys():
    t, c = tc.split("\t")
    if tc_counter[tc] >= 10:
        matrix_x[ordered_t_index[t], ordered_c_index[c]] = max(math.log((N * tc_counter[tc]) / (t_counter[t] * c_counter[c])), 0)

In [41]:
joblib.dump(matrix_x, "matrix_x.pcl", compress=3)
joblib.dump(ordered_t_index, "ordered_t_index.pcl", compress=3)
joblib.dump(ordered_c_index, "ordered_c_index.pcl", compress=3)

['ordered_c_index.pcl']

## 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [42]:
from sklearn.decomposition import TruncatedSVD

In [43]:
matrix_x = matrix_x.tocsr()

In [44]:
truncated_svd = TruncatedSVD(300)

In [45]:
vectors = truncated_svd.fit_transform(matrix_x)

## 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [46]:
vectors[ordered_t_index["United_States"]]

array([ 2.96037375e+00, -9.42029649e-01,  2.63571201e-01,  5.53874168e+00,
        3.04979898e+00, -1.46944730e+00, -7.49094420e-01, -1.67205209e+00,
       -6.14267613e-01, -6.04453060e-01,  2.31791195e-02,  1.13384116e+00,
       -9.15321109e-01, -1.41221918e+00, -8.61060732e-01,  1.15377350e+00,
        1.10576071e+00,  1.12141075e+00, -4.39001067e+00, -1.09708839e+00,
       -2.85223890e-01,  8.16317813e-01,  6.56072030e-03, -1.64780144e-01,
        2.31237454e+00,  1.34190585e+00,  1.73915547e+00, -2.54589787e-01,
       -1.63157455e-01, -3.75725397e-01, -5.46964301e-01,  3.01143309e+00,
       -3.06435040e+00, -1.35201791e+00, -2.64528595e+00,  1.34145373e+00,
        5.77800957e-02,  4.87630482e-02,  1.12125236e+00, -2.31292363e-01,
       -1.21426452e+00, -2.57420718e-02, -1.62385738e+00, -1.25133781e+00,
        1.29856404e+00, -1.25999451e+00, -6.23899254e-03,  5.60531937e-01,
       -9.92199323e-01, -3.55572020e-01,  1.39557672e+00, -8.25212051e-01,
        8.96751098e-01, -

## 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [47]:
from scipy.spatial.distance import cosine

In [48]:
def cosine_sim(v1, v2):
    return max(1- cosine(v1, v2), 0)

In [49]:
cosine_sim(vectors[ordered_t_index["United_States"]], vectors[ordered_t_index["U.S"]])

0.8132832724828232

## 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [50]:
def similary_n_words(target_word, n=10):
    target_vector = vectors[ordered_t_index[target_word]]
    sorted_results = sorted([(t, cosine_sim(target_vector, vectors[ordered_t_index[t]])) for t in ordered_t_index.keys() if t != target_word], key=lambda x: x[1], reverse=True)
    return sorted_results[:n]

In [51]:
similary_n_words("England")

/usr/local/lib/python3.7/site-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


[('France', 0.5558261533890336),
 ('central', 0.2923368410455002),
 ('represented', 0.24200896267723027),
 ('world', 0.22837274579565348),
 ('outside', 0.2270432074435229),
 ('United_States', 0.21260347582012584),
 ('first', 0.17618781035511588),
 ('William', 0.15408748939550254),
 ('revolt', 0.15036620090691621),
 ('next', 0.1473028345728261)]

## 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [52]:
vector = vectors[ordered_t_index["Spain"]] - vectors[ordered_t_index["Madrid"]] + vectors[ordered_t_index["Athens"]]

In [53]:
def similary_n_words_by_vector(target_vector, n=10):
    sorted_results = sorted([(t, cosine_sim(target_vector, vectors[ordered_t_index[t]])) for t in ordered_t_index.keys()], key=lambda x: x[1], reverse=True)
    return sorted_results[:n]

In [54]:
similary_n_words_by_vector(vector)

[('France', 0.5363502333887389),
 ('revolt', 0.31057984074110245),
 ('represented', 0.2609437904081817),
 ('participated', 0.207322799146068),
 ('Wilhelm', 0.20676002584934305),
 ('propaganda', 0.20296480553577556),
 ('self-defense', 0.18553646262420875),
 ('during', 0.15654757237009087),
 ('United_States', 0.1557120026323402),
 ('communism', 0.15285970374512603)]

In [55]:
vector = vectors[ordered_t_index["Spain"]] - vectors[ordered_t_index["Madrid"]]
similary_n_words_by_vector(vector)

[('France', 0.5679562757908425),
 ('revolt', 0.30344814337730663),
 ('represented', 0.24286264011781333),
 ('Wilhelm', 0.23526887766523186),
 ('propaganda', 0.19632402696040463),
 ('during', 0.16685893588231693),
 ('United_States', 0.15411833035437483),
 ('William', 0.1450442552256489),
 ('measures', 0.141387106961254),
 ('attack', 0.13418359484346687)]